<a href="https://colab.research.google.com/github/Rajdeep7102/Rajdeep7102/blob/main/VGG_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/Colab Notebooks'

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
import zipfile 
def unzip_data(filename):
  zip_ref = zipfile.ZipFile(filename,"r")
  zip_ref.extractall()
  zip_ref.close()

In [ ]:
unzip_data('fruits.zip')

In [ ]:
train_dir = "/content/drive/MyDrive/Colab Notebooks/fruits-360_dataset/fruits-360/Training"
test_dir = "/content/drive/MyDrive/Colab Notebooks/fruits-360_dataset/fruits-360/Test"

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
tf.random.set_seed(42)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32,
                                               class_mode="categorical",
                                               seed=42)

test_data = test_datagen.flow_from_directory(test_dir,
                                             batch_size=32,
                                             class_mode="categorical",
                                             seed=42)

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [ ]:
base_model = tf.keras.applications.VGG16(input_shape=(224,224,3),
include_top = False,
weights='imagenet')

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(512,activation="relu")(x)
x=tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(131,activation='softmax')(x)
model = tf.keras.models.Model(base_model.input,x)
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
history_1 = model.fit(train_data,validation_data=test_data,epochs=5)

Epoch 1/5
2116/2116 [==============================] - 572s 270ms/step - loss: 0.3758 - accuracy: 0.8979 - val_loss: 0.3535 - val_accuracy: 0.9133
Epoch 2/5
2116/2116 [==============================] - 580s 274ms/step - loss: 0.1890 - accuracy: 0.9462 - val_loss: 0.2960 - val_accuracy: 0.9260
Epoch 3/5
2116/2116 [==============================] - 576s 272ms/step - loss: 0.1220 - accuracy: 0.9637 - val_loss: 0.2706 - val_accuracy: 0.9371
Epoch 4/5
2116/2116 [==============================] - 580s 274ms/step - loss: 0.0896 - accuracy: 0.9730 - val_loss: 0.2499 - val_accuracy: 0.9417
Epoch 5/5
2116/2116 [==============================] - 587s 277ms/step - loss: 0.0720 - accuracy: 0.9770 - val_loss: 0.2802 - val_accuracy: 0.9422


In [ ]:
model.save("Fruit_classification_VGG16.h5")

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:

# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224):
  """
  Reads an image from filename, turns it into a tensor
  and reshapes it to (img_shape, img_shape, colour_channel).
  """
  # Read in target file (an image)
  img = tf.io.read_file(filename)

  # Decode the read file into a tensor & ensure 3 colour channels 
  # (our model is trained on images with 3 colour channels and sometimes images have 4 colour channels)
  img = tf.image.decode_image(img, channels=3)

  # Resize the image (to the same size our model was trained on)
  img = tf.image.resize(img, size = [img_shape, img_shape])

  # Rescale the image (get all values between 0 and 1)
  img = img/255.
  return img

In [ ]:
import matplotlib.pyplot as plt
def pred_and_plot(model, filename, class_names):
  """
  Imports an image located at filename, makes a prediction on it with
  a trained model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  pred_class = class_names[int(tf.round(pred)[0][0])]

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

In [ ]:
apple = load_and_prep_image("/content/drive/MyDrive/Colab Notebooks/fruits-360-original-size/fruits-360-original-size/Test/apple_braeburn_1/r0_103.jpg")
apple

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       ...,

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)>

NameError: ignored

In [ ]:
best_pred = tf.keras.applications.vgg16.decode_predictions(pred_img)

ValueError: ignored

In [ ]:
image = tf.keras.preprocessing.image.load_img('/content/drive/MyDrive/Colab Notebooks/fruits-360-original-size/fruits-360-original-size/Test/apple_braeburn_1/r0_103.jpg', target_size=(224, 224))
# convert the image pixels to a numpy array
image = tf.keras.preprocessing.image.img_to_array(image)
# reshape data for the model
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# prepare the image for the VGG model
image = tf.keras.applications.vgg16.preprocess_input(image)
# predict the probability across all output classes
yhat = model.predict(image)
# convert the probabilities to class labels
label = tf.keras.applications.vgg16.decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
label = label[0][0]
# print the classification
print('%s (%.2f%%)' % (label[1], label[2]*100))

1/1 [==============================] - 0s 17ms/step


ValueError: ignored

In [ ]:
import tensorflow as tf 
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Fruit_classification_VGG16.h5')

In [ ]:
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
pred_img = new_model.predict(tf.expand_dims(apple, axis=0))

1/1 [==============================] - 10s 10s/step


In [ ]:
pred_class_index = np.argmax(pred_img)

In [ ]:
class_label = {'Apple Braeburn',
 'Apple Crimson Snow',
 'Apple Golden 1',
 'Apple Golden 2',
 'Apple Golden 3',
 'Apple Granny Smith',
 'Apple Pink Lady',
 'Apple Red 1',
 'Apple Red 2',
 'Apple Red 3',
 'Apple Red Delicious',
 'Apple Red Yellow 1',
 'Apple Red Yellow 2',
 'Apricot',
 'Avocado',
 'Avocado ripe',
 'Banana',
 'Banana Lady Finger',
 'Banana Red',
 'Beetroot',
 'Blueberry',
 'Cactus fruit',
 'Cantaloupe 1',
 'Cantaloupe 2',
 'Carambula',
 'Cauliflower',
 'Cherry 1',
 'Cherry 2',
 'Cherry Rainier',
 'Cherry Wax Black',
 'Cherry Wax Red',
 'Cherry Wax Yellow',
 'Chestnut',
 'Clementine',
 'Cocos',
 'Corn',
 'Corn Husk',
 'Cucumber Ripe',
 'Cucumber Ripe 2',
 'Dates',
 'Eggplant',
 'Fig',
 'Ginger Root',
 'Granadilla',
 'Grape Blue',
 'Grape Pink',
 'Grape White',
 'Grape White 2',
 'Grape White 3',
 'Grape White 4',
 'Grapefruit Pink',
 'Grapefruit White',
 'Guava',
 'Hazelnut',
 'Huckleberry',
 'Kaki',
 'Kiwi',
 'Kohlrabi',
 'Kumquats',
 'Lemon',
 'Lemon Meyer',
 'Limes',
 'Lychee',
 'Mandarine',
 'Mango',
 'Mango Red',
 'Mangostan',
 'Maracuja',
 'Melon Piel de Sapo',
 'Mulberry',
 'Nectarine',
 'Nectarine Flat',
 'Nut Forest',
 'Nut Pecan',
 'Onion Red',
 'Onion Red Peeled',
 'Onion White',
 'Orange',
 'Papaya',
 'Passion Fruit',
 'Peach',
 'Peach 2',
 'Peach Flat',
 'Pear',
 'Pear 2',
 'Pear Abate',
 'Pear Forelle',
 'Pear Kaiser',
 'Pear Monster',
 'Pear Red',
 'Pear Stone',
 'Pear Williams',
 'Pepino',
 'Pepper Green',
 'Pepper Orange',
 'Pepper Red',
 'Pepper Yellow',
 'Physalis',
 'Physalis with Husk',
 'Pineapple',
 'Pineapple Mini',
 'Pitahaya Red',
 'Plum',
 'Plum 2',
 'Plum 3',
 'Pomegranate',
 'Pomelo Sweetie',
 'Potato Red',
 'Potato Red Washed',
 'Potato Sweet',
 'Potato White',
 'Quince',
 'Rambutan',
 'Raspberry',
 'Redcurrant',
 'Salak',
 'Strawberry',
 'Strawberry Wedge',
 'Tamarillo',
 'Tangelo',
 'Tomato 1',
 'Tomato 2',
 'Tomato 3',
 'Tomato 4',
 'Tomato Cherry Red',
 'Tomato Heart',
 'Tomato Maroon',
 'Tomato not Ripened',
 'Tomato Yellow',
 'Walnut',
 'Watermelon'}

In [ ]:
type(class_label)

set

In [ ]:
pred_class_index

27

In [ ]:
predictions_reshaped = pred_img.reshape(1, 1000)

# Decode the predictions
predictions_decoded = tf.keras.applications.vgg16.decode_predictions(predictions_reshaped)


ValueError: ignored

In [ ]:
# Decode the predictions
#predictions = tf.keras.applications.vgg16.decode_predictions(pred_img)
predictions=[pred_img]
print(predictions)
# Map the probabilities to their respective class names
for prediction in predictions:
  class_name = "Unknown"
  if prediction[0] in class_label:
    class_name = class_labels[prediction[0].tolist()]
  probability = prediction[1]
  print('Predicted:', class_name, 'with probability', probability)

[array([[2.70692224e-04, 2.82984167e-01, 1.19236053e-11, 3.71966416e-05,
        5.14131889e-14, 8.63519054e-13, 3.50589071e-05, 6.49403034e-07,
        4.24886126e-14, 7.44028097e-28, 4.41427028e-09, 3.47271264e-02,
        2.15444122e-15, 1.12949527e-17, 7.10430158e-22, 1.76638720e-19,
        1.76005307e-17, 1.99627066e-05, 6.17148867e-18, 3.50617460e-18,
        6.63274907e-17, 8.06630829e-21, 1.21627548e-13, 1.39018800e-25,
        9.75960415e-13, 2.30095745e-19, 7.77842255e-13, 4.41280663e-01,
        1.74158465e-11, 2.38205803e-15, 4.69723652e-13, 9.25724481e-21,
        3.09313880e-10, 3.18682357e-03, 1.09490472e-29, 2.81594333e-21,
        1.72435414e-22, 1.34946159e-13, 8.54525615e-17, 4.65060640e-23,
        4.69235540e-17, 9.76331549e-08, 2.17471651e-19, 1.38873053e-17,
        1.42253621e-18, 2.22861595e-12, 1.04930567e-16, 9.92223659e-18,
        2.89089355e-28, 4.71871198e-22, 1.36817775e-07, 5.58117784e-18,
        1.03918209e-19, 1.69086543e-16, 2.62600027e-13, 9.57468

TypeError: ignored